<a href="https://colab.research.google.com/github/erobrien6/OBrien_DSPN_S23/blob/main/Exercise12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross Validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes. 

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [44]:
# INSERT CODE HERE
#install.packages('boot')
#install.packages('mlbench')

library(tidyverse)
library(boot)
library(mlbench)

# Read in data
data(PimaIndiansDiabetes2)
df_pima <- PimaIndiansDiabetes2

df_pima <- df_pima %>%
  select(c(1:4, 6:9))

# Drop NAs
nrow(df_pima)
df_final <- drop_na(df_pima)
nrow(df_final) # Check that drop worked

head(df_final)
dim(df_final)


[1] 768

[1] 532

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


[1] 532   8

(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset. 

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%. 

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [45]:
# Logistic Regression
# Subset data to exlude first row
df_model <- df_final[-1,]
nrow(df_model)

log_fit <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age, data = df_model, family = binomial)
summary(log_fit)

# Predict diabetes

df_model$diabetes <- as.factor(df_model$diabetes)
new_case = df_final[1,]
test <- predict(log_fit, new_case, type = "response")
test

# The outcome was 0.73, a positive number, meaning that there is above 50% chance
# that the model classified this case as positive for diabetes



[1] 531


Call:
glm(formula = diabetes ~ pregnant + glucose + pressure + triceps + 
    mass + pedigree + age, family = binomial, data = df_model)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.0041  -0.6633  -0.3704   0.6439   2.4793  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -9.537281   0.993617  -9.599  < 2e-16 ***
pregnant     0.122949   0.043708   2.813 0.004909 ** 
glucose      0.035231   0.004240   8.308  < 2e-16 ***
pressure    -0.007469   0.010314  -0.724 0.468963    
triceps      0.006572   0.014744   0.446 0.655779    
mass         0.082720   0.023320   3.547 0.000389 ***
pedigree     1.304616   0.363719   3.587 0.000335 ***
age          0.025756   0.014017   1.838 0.066133 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 674.58  on 530  degrees of freedom
Residual deviance: 465.70  on 523  degrees of freedom
AIC: 481.7

N

1 
0.729487

So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row. 

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [46]:
# Initialize `results` data frame
results <- data.frame(actual = df_final$diabetes, predicted = NA)

#for loop
for (i in 1:nrow(df_final)){ 
    # separate individual observation `i` from the rest of your data
    df_train <- df_final[-i,]
    
    # train your model
    log_fit2 <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age, data = df_train, family = binomial)
    
    # test model on hold out observation
    df_train$diabetes <- as.factor(df_train$diabetes)
    new_case = df_final[i,]
    result = predict(log_fit2, new_case, type = "response") 
    
    # classify model prediction as "pos" or "neg" and add to `results`
    if(result >= 0.5){
      pred = "pos"
    } else {
      pred = "neg"
    }
    results[i, 'predicted'] = pred
    
}
head(results)

,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [47]:
correct = 0
total = 0
for (i in 1:nrow(results)){
  total = total + 1
  if(results[i, 'actual'] == results[i, 'predicted']){
    correct = correct + 1
  }
}

error = 1 - (correct/total)
print(paste("Overall Error:", error)) #0.222


[1] "Overall Error: 0.221804511278195"


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [48]:
#?cv.glm
cv_model = cv.glm(df_model, log_fit, K=nrow(df_model))
cv_model$delta



[1] 0.1473113 0.1473056

Here, we see `cost` is defined as: 
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is 

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [49]:
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
cv_model = cv.glm(df_model, log_fit, cost, K=nrow(df_model))
cv_model$delta

#0.224 unadj, 0.225 adj

[1] 0.2241055 0.2250559

How do your results compare to your manual LOOCV above?

> * *The manual LOOCV seems to have been slightly better at correctly classifying diabetes cases because its error was 0.222, whereas the cv.glm error was slightly higher at 0.225.*

# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15. 

In [54]:
set.seed(1)
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)

# K = 3
cv_model1 = cv.glm(df_model, log_fit, cost, K=3)
cv_model1$delta

# K = 5
cv_model2 = cv.glm(df_model, log_fit, cost, K=5)
cv_model2$delta

# K = 10
cv_model3 = cv.glm(df_model, log_fit, cost, K=10)
cv_model3$delta

# K = 15
cv_model4 = cv.glm(df_model, log_fit, cost, K=15)
cv_model4$delta



[1] 0.2109228 0.2153170

[1] 0.2222222 0.2244779

[1] 0.2203390 0.2243218

[1] 0.2259887 0.2283153

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * *For k less than or equal to 5, the error is smaller than the LOOCV error above (0.225). However, as k increases, the error continues to increase in small increments, so k = 3 is the best choice for increasing the model fitting accuracy here.*
> 


If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> * *I would expect the results to stay consistent for the LOOCV. For the random seed above, the seed sets the randomization starting point of the models, so there could be more variation in classification (and more variation in error) when k is lower versus higher. When k is maxxed out (LOOCV), we are running our model iteratively on one case at a time, and the model is trained on every data point besides the one excluded, so the error should stay consistent.*
> 


**DUE:** 5pm March 27, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>

